In [1]:
# 之前的分析一直是词袋分析，没有把句子的先后顺序反应出来
# 然后看了一些博客，paper，sentence2vec虽然也有，但不过是用word2vec的方法在句子层面上做了一遍
# 也有一些基于word2vec+rnn的句子相似度模型，其实这个挺不错，但是我目前的tensorflow基础好像还做不到
# 最终决定先做一个word2vec，然后用词向量的简单平均求和看看，顺便也复习一下tensorflow里怎么实现word2vec

In [3]:
import pandas as pd
import numpy as np
filepath = 'C:/Users\Administrator\Desktop/'

In [4]:
train_data = pd.read_csv(filepath+'train.csv')
test_data = pd.read_csv(filepath+'test.csv')

In [18]:
data_combine = pd.concat([train_data,test_data],axis=0)

In [19]:
from nltk import WordNetLemmatizer
from nltk import PorterStemmer
from nltk import word_tokenize

In [20]:
data_combine['word_split'] = data_combine.text.apply(lambda x:word_tokenize(str(x)))

In [24]:
wordlist = []
for i in data_combine.word_split:
    for x in i:
        if x not in wordlist:
            wordlist.append(x)

In [26]:
vocab_size = len(wordlist)

In [28]:
# 建立两个字典
word2int = {}
int2word = {}
for i,i_index in enumerate(wordlist):
    word2int[i]=i_index
    int2word[i_index] = i

In [36]:
import tensorflow as tf
import math

In [261]:
embed_size = 100
batch_size = None
win = 3
epoch = 60000
n=0
pp=[[],[]]

In [262]:
graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder(shape=[batch_size],dtype=tf.int32)
    labels = tf.placeholder(shape=[batch_size,1],dtype=tf.int32)
    embed_dict = tf.Variable(
        tf.random_uniform(shape=[vocab_size,embed_size],minval=-1.0,maxval=1.0,name='vector')
    )
    nce_weight = tf.Variable(
        tf.truncated_normal(shape=[vocab_size,embed_size],stddev=1.0/math.sqrt(embed_size))
    )
    nce_bias = tf.Variable(tf.zeros([vocab_size]))
        
    embed = tf.nn.embedding_lookup(embed_dict,inputs)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weight,biases=nce_bias,labels=labels,num_sampled=100,num_classes=vocab_size,inputs=embed)
    )
    train_op = tf.train.GradientDescentOptimizer(learning_rate=0.04).minimize(loss)     
    init = tf.global_variables_initializer()

In [263]:
def train(input_sentence,n=n):
        batch_input = []
        batch_label = []
        sent_len = len(input_sentence)
        for i in range(sent_len):
            start = max(0,i-win)
            end = min(sent_len,i+win)
            for index in range(start,end):
                if index==i:
                    continue
                else:
                    input_id = int2word[input_sentence[i]]
                    label_id = int2word[input_sentence[index]]
                    batch_input.append(input_id)
                    batch_label.append(label_id)
        if len(batch_input)==0:
            return
        batch_input = np.array(batch_input,dtype=np.int32)
        batch_label = np.array(batch_label,dtype=np.int32)
        batch_label = np.reshape(batch_label,[len(batch_label),1])
        feed_dic = {inputs:batch_input,labels:batch_label}
        _,loss_val = sess.run([train_op,loss],feed_dict=feed_dic)
        n+=1
        if n%100==0:
            pp[0].append(n)
            pp[1].append(loss_val)
        return n

In [264]:
sess = tf.Session(graph=graph)
sess.run(init)
with sess.as_default():
    for j in range(2):
        for i in data_combine.word_split:
            if n<epoch:
                n = train(i,n)
    final_embeddings = embed_dict.eval()

In [268]:
final_embed = pd.DataFrame(final_embeddings,columns=['embed_'+str(i) for i in range(embed_size)])

In [270]:
def embed(x,num):
    sum = 0
    l = 0
    for i in x:
        index = int2word[i]
        sum+=final_embed['embed_'+str(num)][index]
        l+=1
    return sum/l
for i in range(embed_size):
    data_combine['embed_mean'+str(i)] = data_combine.word_split.apply(lambda x:embed(x,i))
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [272]:
writer = pd.ExcelWriter('D:/大学/大三上/数据挖掘/作业/数据embed.xlsx')
data_combine.to_excel(writer,sheet_name='sheet1')
writer.save()